In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [3]:
%load_ext autoreload
%autoreload 2
from scripts.metrics.summary_evaluation import SummaryEvaluation
from scripts.models.rank_with_model.rank_with_model import RankModel
from scripts.models.ltr_nn_tf.train import LTRNNTFTrain
from scripts import conf
%reload_ext autoreload

In [4]:
import pandas as pd

In [53]:
ltr_params = {
    'mode': 'tfidf',
    'count_vec_kwargs': {'strip_accents': 'unicode'},
    'target_metric': 'rouge',
    'metric_params': {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'},
    'drop_teams': True,
    'lemma': True,
    'train_perc': 0.7,
    'val_perc': 0.2
}
DENSE_LAYERS = [500, 250, 125, 60, 20]
N_DROPOUT = 4

model_params = {
    'dense_layers': DENSE_LAYERS,
    'dense_activation': ['relu'] * len(DENSE_LAYERS),
    'dropout': [0.3] * N_DROPOUT,
    'optimizer': 'adam'
}
opt_metric = 'mean_squared_error'
epochs = 25
batch_size = 128
# Not in config
shuffle = True
max_queue_size = 50

In [54]:
ltr_train = LTRNNTFTrain(ltr_params, epochs=epochs, batch_size=batch_size,
                         model_params=model_params,
                         opt_metric=opt_metric)

Adding 1 zeros so that params have the same length
Setting target metric to rouge
Setting mode to tfidf
Optimizing mean_squared_error with adam
Epochs: 25
Batch size: 128


In [59]:
rank_model = RankModel(ltr_train, n=10, is_nn=True)

Model already trained


In [15]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/254dd4d848'

Si queremos usar una csr matrix en la red, necesita que los índices estén
ordenados. Los índices apuntan a los valores que no son cero

In [17]:
features_target = rank_model.load_data()

In [26]:
x = features_target[0]
y = features_target[1]

In [25]:
x.indices

array([5593, 5573, 4684, ...,  877,  434,  424], dtype=int32)

In [42]:
docs = ['hola carlos', 'hola isabel', 
       'hola carlos', 'hola maria', 'hola maria', 
       'hola carlos']

In [43]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline

In [44]:
count_vec_kwargs = {'strip_accents': 'unicode'}

In [45]:
pipe = Pipeline([('count', CountVectorizer(**count_vec_kwargs)),
                             ('tfidf', TfidfTransformer())])

In [46]:
x_t = pipe.fit_transform(docs)

In [47]:
x_t

<6x4 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [48]:
x_t.indices

array([1, 0, 2, 1, 1, 0, 3, 1, 3, 1, 1, 0], dtype=int32)

In [49]:
pd.DataFrame(x_t.toarray(),columns=pipe['count'].get_feature_names())

,carlos,hola,isabel,maria
0,0.841819,0.539760,0.000000,0.000000
1,0.000000,0.405722,0.913996,0.000000
2,0.841819,0.539760,0.000000,0.000000
3,0.000000,0.476055,0.000000,0.879415
4,0.000000,0.476055,0.000000,0.879415
5,0.841819,0.539760,0.000000,0.000000


In [36]:
x_t.toarray()

array([[0.52064623, 0.43218341, 0.        , 0.52064623, 0.52064623,
        0.        , 0.        ],
       [0.        , 0.40382593, 0.60298477, 0.48648432, 0.48648432,
        0.        , 0.        ],
       [0.70710678, 0.        , 0.        , 0.        , 0.        ,
        0.70710678, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.62791376, 0.77828292],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [50]:
x_t.sort_indices()

Solo ordena los indices dentro de cada fila, por lo que cada fila 
se mantiene en su sitio.

In [52]:
x_t.indices

array([0, 1, 1, 2, 0, 1, 1, 3, 1, 3, 0, 1], dtype=int32)

In [38]:
x_t.toarray()

array([[0.52064623, 0.43218341, 0.        , 0.52064623, 0.52064623,
        0.        , 0.        ],
       [0.        , 0.40382593, 0.60298477, 0.48648432, 0.48648432,
        0.        , 0.        ],
       [0.70710678, 0.        , 0.        , 0.        , 0.        ,
        0.70710678, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.62791376, 0.77828292],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [40]:
pd.DataFrame(x_t.toarray(),columns=pipe['count'].get_feature_names())

,carlos,hola,isabel,llamo,me,soy,tonto
0,0.520646,0.432183,0.000000,0.520646,0.520646,0.000000,0.000000
1,0.000000,0.403826,0.602985,0.486484,0.486484,0.000000,0.000000
2,0.707107,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.627914,0.778283
4,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [57]:
type(x_t)

scipy.sparse.csr.csr_matrix